# Visualizing Our Curriculum

In [3]:
%%html
<div id="d3-bioheader"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>


I was interested in how various courses relate to each other within departments, and between departments. I also wanted to know things like how many service courses each department teaches. As far as I know, my school doesn't keep a nice electronic version of the course guide with unique IDs per course along with lists of dependencies, etc. So, the plan was

 1. Scrape our website for a list of courses taught by each department as well as a list of major and minor requirements.
 1. Do a passable (but not perfect) job of cleaning the data, then stuff it all into a graph.
 1. Add it up. And graph things, using this as an opportunity to learn a bit more [D3](http://d3js.org/).
 
I only had a couple of evenings worth of free time, so it's not perfect, but it's pretty good, at least for the science division.

Current Issues

 * Does not understand "N courses numbered XXX and above" so doesn't understand that PHYS 225 counts towards BIO minor.
 * For service courses, it would be useful to know which courses are *required* by other departments, rather than which courses other departments are willing to count towards a major/minor. We could do this by adding weights or colors to the edges.
 * When the prerequisites include "or consent of the instructor" I assume you will never get that consent.
 * Does not understand less structured requirements, e.g. "Prerequisites: Sophomore standing or above and either one Philosophy course or one Women's Studies course."
 * Understands requirements (A-AR, D-D, etc.) but doesn't do anything with them.
 * Does not understand lab courses (easy to fix).
 * Does not understand cross-listed courses.
 * Now I have two problems.

I haven't yet figured out how to migrate my wordpress [blog](http://mglerner.com/blog) to IPython Notebooks while preserving all of the comments. When I do, though, I may switch over to Disqus. So, you might ask, why not just stick Disqus comments at the end of a notebook and stick it on github/nbreader? Why not indeed.

Anyway, first things first, let's load up our libraries.

In [5]:
from __future__ import division
import numpy as np, scipy as sp, pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import urllib, re, json, csv
from bs4 import BeautifulSoup
from networkx.readwrite import json_graph

#%install_ext https://raw.githubusercontent.com/rasbt/watermark/master/watermark.py
#%load_ext watermark
#%watermark -v -m -p numpy,pandas,networkx,scipy
%matplotlib inline

## Code at the bottom

Since this is meant to be readable like a blog entry, I've put most of the code at the bottom. If you're using it like a notebook, scroll down to the end and execute all of the code cells there first. After you've done that, you get a couple of useful functions:

 * `getcourses(dept,url,verbose=False)` which grabs data from our website and extracts the course information as a graph. A few things to note here:
  * I use a directed graph (`nx.DiGraph`) so that we can show which courses depend on which other courses.
  * I hear that all of the cool kids are using [scrapy](http://scrapy.org/) these days, but I couldn't figure out how to make scrapy play with the HTML structure of our pages. I used the old standby [Beautiful Soup](http://www.crummy.com/software/BeautifulSoup/) instead, along with a few regular expressions.
  * As mentioned above, this works well with Science Division courses, but not so well with many other courses. So, I'll stick to SciDiv in this post.
 * `plotdept(...)` will use `networkx` to plot things. I think the D3 plots look nicer, so I've just used those in this post. They're a bit more cumbersome to write, though, so `plotdept` is still useful.
 * `mgDiGcompose(g,h)` I parse each department separately and then compose the graphs. The standard `nx.compose` will overwrite nodes in `h` with information from nodes in `g`. That causes me trouble (e.g. BIOL (`g`) requires PHYS 235, but PHYS (`h`) has already defined the course. All of the information in `h` (whether PHYS 235 is an A-AR, etc.) gets lost. `mgDiGcompose` composes two graphs, but preserves as many node attributes as possible.
 * `fudgerowpart(...)` is what we use to try to guess course attributes just from the name.
 * `addedges(g,h)` if there's a edge in `g` that corresponds to nodes in `h`, or vice versa, it gets added to the other graph. For instance, the Bio

So let's scrape our website and stuff everything into a dictionary where we can look up by department names.
  

In [6]:
gs = {}
deptandurl = [('PHYS','http://www.earlham.edu/physics-and-astronomy/the-program/'),
              ('MATH','http://www.earlham.edu/mathematics/the-program/'),
              ('BIOL','https://www.earlham.edu/biology/the-program/'),
              ('CHEM','http://www.earlham.edu/chemistry/the-program/'),
              ('GEOL','http://www.earlham.edu/geology/the-program/'),
              ('CS','https://www.earlham.edu/computer-science/the-program/'),
              ('BIOC','http://www.earlham.edu/biochemistry/the-program/')
              #('PHIL','http://www.earlham.edu/philosophy/the-program/'), # PHIL 488 depends on PHIL 488 really?
              #('ANCS','http://www.earlham.edu/ancient-and-classical-studies/the-program/',)
              ]
for (dept,url) in deptandurl:
    gs[dept] = getcourses(dept,url)
allg = gs[deptandurl[0][0]]
for (dept,url) in deptandurl[1:]:
    allg = mgDiGcompose(allg,gs[dept])
for (dept,g) in gs.items():
    addedges(allg,g)

NameError: name 'getcourses' is not defined

You can see a few errors, but overall, we got most of the data. The 'CS310' error is because it should be called 'CS 310'. I've dealt with quite a few formatting errors (see the code at the end), but not all of them.

## Service Courses

This is a tricky question to answer in full. I'll just ask which courses other departments are willing to count towards their major/minor. Note that most of the Biology and Chemistry service classes are services to the Biochemistry major.

In [5]:
servicecourses = {}
allservicecourses = {}
for d1 in gs:
    allservicecourses[d1] = {}
    for d2 in gs:
        allservicecourses[d1][d2] = []
def tonumlist(s):
    return ' '.join([str(j) for j in sorted([int(i[-3:]) for i in s])])
sg = nx.DiGraph()
for dept in gs:
    thisg = gs[dept]
    othergs = [(i,gs[i]) for i in gs if i != dept]
    service = set()
    for (od,og) in othergs:
        thisservice = set()
        for c in og.nodes():
            if c.startswith(dept):
                service.add(c)
                thisservice.add(c)
        sg.add_edge(od,dept,weight=len(thisservice))
        allservicecourses[dept][od] = tonumlist(thisservice)
        #print("I think {d} {od} {nl}".format(d=dept,od=od,nl=tonumlist(thisservice)))
    servicecourses[dept] = service
for dept in servicecourses:
    scs = tonumlist(servicecourses[dept])
    print("{d:4} provides {n:2}: {s} total".format(d=dept,n=len(servicecourses[dept]),s=scs))
    for od in allservicecourses[dept]:
        if allservicecourses[dept][od]:
            print("        for {od:4}: {s}".format(od=od,s=allservicecourses[dept][od]))

GEOL provides  1: 211 total
        for BIOL: 211
CHEM provides 10: 111 221 321 331 341 351 361 371 431 480 total
        for GEOL: 111 221 331
        for BIOC: 111 221 321 331 341 351 361 371 431 480
        for PHYS: 111 331
        for BIOL: 111 221 321 331
BIOC provides  0:  total
PHYS provides  6: 120 125 220 225 230 235 total
        for GEOL: 120 125
        for CHEM: 120 125 220 225 230 235
        for BIOC: 120 125 230 235
        for BIOL: 120 125
        for CS  : 230 235
BIOL provides 14: 111 112 226 341 343 345 347 460 461 462 464 465 466 480 total
        for GEOL: 111
        for BIOC: 112 226 341 343 345 347 460 461 462 464 465 466 480
MATH provides  8: 120 180 190 195 280 320 350 360 total
        for GEOL: 120 180
        for CHEM: 180
        for BIOC: 120 180
        for PHYS: 180 280 320 350 360
        for BIOL: 120 180 280
        for CS  : 180 190 195
CS   provides  1: 128 total
        for PHYS: 128


## Plotting the curricula

In [6]:
#plotdept(allg,layout='spring',layoutparams={'k':10.0, 'iterations':100}, figsize=(20,20))
#plotdept(gs['BIOL'],layout='spring', autocolor=True, layoutparams={'iterations':25}, figsize=(25,25))
#plotdept(allg,drawparams={},layout='spring',layoutparams={'iterations':300}, figsize=(25,25),autocolor=True)

I like [this](http://bl.ocks.org/mbostock/4062045) force-directed graph, which we can mimic via some [IPython Cookbook](http://ipython-books.github.io/) [code](http://nbviewer.ipython.org/github/ipython-books/cookbook-code/blob/master/notebooks/chapter06_viz/04_d3.ipynb) and [this](http://bl.ocks.org/d3noob/5155181) directed graph, which we can mimic by actually understanding a tiny bit of D3 ourselves. Overall, I like the directed one a bit better, but I've included code for the undirected one if you want to try it.

For the demo-like force-directed graph, we stick the data into a json file that we can slurp back up in javascript.
For the directed graph, we'll use both a comma-separated and tab-separated file, one per department. The parsing is not perfect here, and I've left in the error messages so as not ot be misleading.

In [7]:
d = json_graph.node_link_data(allg)
json.dump(d, open('curriculum.json','w'))
#g = gs['PHYS']
g = allg

with open('curriculum-links.csv','w') as f:
    f.write('source,target,value\n')
    for (source,target) in g.edges():
        f.write('{s},{t},{v}\n'.format(s=source,t=target,v=1.0))
with open('curriculum-nodes.tsv','w') as f:
    writer = csv.writer(f,delimiter='\t')
    parts = 'label dept num satisfies prereqs coreqs ay'.split()
    writer.writerow(parts)
    for n in g.nodes():
        #print('{n} {gn}'.format(n=n,gn=g.node[n]))
        rowparts = []
        #writer.writerow([g.node[n][p] for p in parts])      
        for p in parts:
            try:
                rowparts.append(g.node[n][p])
            except KeyError:
                print("Could not find {p} in {n}".format(p=p,n=n))
                
                rowparts.append(fudgerowpart(p,n))
        writer.writerow(rowparts)
for dept in gs:
    print("\n\nDOING {d}\n\n".format(d=dept))
    g = gs[dept]
    with open('curriculum-links-{d}.csv'.format(d=dept),'w') as f:
        f.write('source,target,value\n')
        for (source,target) in g.edges():
            f.write('{s},{t},{v}\n'.format(s=source,t=target,v=1.0))
    with open('curriculum-nodes-{d}.tsv'.format(d=dept),'w') as f:
        writer = csv.writer(f,delimiter='\t')
        parts = 'label dept num satisfies prereqs coreqs ay'.split()
        writer.writerow(parts)
        for n in g.nodes():
            #print('{n} {gn}'.format(n=n,gn=g.node[n]))
            rowparts = []
            #writer.writerow([g.node[n][p] for p in parts])      
            for p in parts:
                try:
                    rowparts.append(g.node[n][p])
                except KeyError:
                    print("Could not find {p} in {n}".format(p=p,n=n))
                    rowparts.append(fudgerowpart(p,n))
            writer.writerow(rowparts)

Could not find dept in PHYS 220
	Fudging PHYS
Could not find num in PHYS 220
	Fudging 220
Could not find satisfies in PHYS 220
Could not find prereqs in PHYS 220
Could not find coreqs in PHYS 220
Could not find ay in PHYS 220
Could not find dept in FACULTY ADVISER'S PRIOR APPROVAL OF PROJECT PROPOSAL
Could not find num in FACULTY ADVISER'S PRIOR APPROVAL OF PROJECT PROPOSAL
Could not find satisfies in FACULTY ADVISER'S PRIOR APPROVAL OF PROJECT PROPOSAL
Could not find prereqs in FACULTY ADVISER'S PRIOR APPROVAL OF PROJECT PROPOSAL
Could not find coreqs in FACULTY ADVISER'S PRIOR APPROVAL OF PROJECT PROPOSAL
Could not find ay in FACULTY ADVISER'S PRIOR APPROVAL OF PROJECT PROPOSAL
Could not find satisfies in PSYC 342
Could not find prereqs in PSYC 342
Could not find coreqs in PSYC 342
Could not find ay in PSYC 342
Could not find dept in SENIOR STANDING
Could not find num in SENIOR STANDING
Could not find satisfies in SENIOR STANDING
Could not find prereqs in SENIOR STANDING
Could not fi

## Individual Departments

The next cell sets up a relatively blank DIV that to which can attach javascript.

Down at the end, you'll see the large javascript cells that set up the actual force-directed graphs. If you're running this yourself, that means you'll have to run the HTML-defining cells first, then scroll down and run the javascript cells.

### Biology
Let's check out Biology first.

You can tell a lot about the major by looking at the graph below. For instance, two things that stand out are

 * There are clearly two "tracks" in the major, with only BIO 112 (Cells, Genes and Inheritance) shared between them.
 * Chemistry and Psychology are explicitly required for some courses, but no Biology courses require explicit Math, Physics, Computer Science or Geology information.
 
The "BOGUS" node below is something I couldn't parse. 

In [3]:
%%html
<div id="d3-bio"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Physics

In [4]:
%%html
<div id="d3-phys"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Mathematics

In [10]:
%%html
<div id="d3-math"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Chemistry

In [11]:
%%html
<div id="d3-chem"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Geology

In [12]:
%%html
<div id="d3-geol"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Computer Science

In [13]:
%%html
<div id="d3-cs"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Biochemistry

In [14]:
%%html
<div id="d3-bioc"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

### Environmental Science

Omitted because the webpage is not structured like other departmental webpages

### Neuroscience

Omitted because the webpage is not structured like other departmental webpages

## The whole division

Now let's look at the entire science division, first as a directed graph as above

In [17]:
%%html
<div id="d3-scidiv"></div>
<style>
path.link {fill: none;  stroke: #666;  stroke-width: 1.5px;}
circle {  fill: #ccc;  stroke: #fff;  stroke-width: 1.5px;}
text {  fill: #000;  font: 10px sans-serif;  pointer-events: none;  stroke: #000;  stroke-width: 0.2px;}
</style>

And second as a force-directed graph in case you hate the arrows.

In [18]:
%%html
<div id="d3-example1"></div>
<style>
.node {stroke: #000; stroke-width: 1.5px;}
.link {stroke: #999; stroke-opacity: .6;}
</style>

## Execute this code first

Run everything below here after running cell 1 if you're doing this interactively.

In [7]:
def getrequiredcourses(dept,url,verbose=False):
    """Just get node names for courses; assume that the Courses page has filled in all of the details.
    """
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page)
    allnames = []
    try:
        courseshref = soup.find('a',text='Plan of Study')['href']
    except TypeError:
        courseshref = soup.find('a',text='Plan of Study ')['href']
    coursetab = soup.find(id=courseshref[1:])
    allp = coursetab.findAll('li')
    # Two problems. Also, see http://stackoverflow.com/questions/18425386/re-findall-not-returning-full-match-python-2-7
    # for an explanation of the ?:. If you use groups, findall gives the groups. Use noncapturing groups to get the text
    coursepat = re.compile('[A-Z]{2,4} [0-9]{3}(?:(?:, | and | or )[0-9]{3})*') 
    depends = []
    for p in allp:
        t = p.text
        matches = coursepat.findall(t)
        if matches:
            if verbose: print("For {t}".format(t=t))
            for match in matches:
                if verbose: print("\tFor {m}".format(m=match))
                match = match.replace(' and ',', ').replace(' or ',', ')
                if len(match.split()) == 2:
                    dept,num = match.split()
                    name = '{dept} {num}'.format(dept=dept,num=num)
                    names = [name]
                    allnames.extend(names)
                else:
                    dept = match.split()[0]
                    nums = match.replace(', ',' ').split()[1:]
                    names = ['{dept} {num}'.format(dept=dept,num=num) for num in nums]
                    allnames.extend(names)
                if verbose: print("\tmatched {n}".format(n=names))
        else:
            print("NO MATCH for {t}".format(t=t))
    return allnames
def getcourses(dept,url,verbose=False):
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page)
    courseshref = soup.find('a',text='Courses')['href']
    coursetab = soup.find(id=courseshref[1:])
    allp = coursetab.findAll('p')
    coursepat = re.compile('.?{dept}.*'.format(dept=dept))
    g = nx.DiGraph()
    depends = []
    for p in allp:
        t = p.text
        if coursepat.match(t):
            num = t.split()[1]
            name = '{dept} {num}'.format(dept=dept,num=num)
            ay = False
            #print(t)
            satisfies = getsatisfies(t)
            prereqs = getprereqs(t,pre_co='Pre')
            coreqs = getprereqs(t,pre_co='Co')
            if verbose:
                print("{name} satisfies {s}. I am {ay}taught every year.".format(name=name,s=satisfies,ay={True:'not ',False:''}[ay]))
                if prereqs: print("I have prereqs {p}".format(p=prereqs))
                if coreqs: print("I have coreqs {p}".format(p=coreqs))
            name,dept = name.upper(),dept.upper()
            satisfies = [i.upper() for i in satisfies]
            prereqs = [i.upper() for i in prereqs]
            coreqs = [i.upper() for i in coreqs]
            g.add_node(name,
                       label=name,
                       satisfies=satisfies,prereqs=prereqs,coreqs=coreqs,ay=ay,
                       num=num,dept=dept)
            for pr in prereqs:
                depends.append([pr,name])
            for cr in coreqs:
                depends.append([cr,name])
    for d in depends:
        g.add_path(d)
    reqs = getrequiredcourses(dept,url,verbose)
    for n in reqs:
        if n not in g.node:
            g.node[n] = {'label':n,
                         'num':n.split()[-1],
                         'dept':n.split()[0],
                         }
    # It's possible to add edges that don't correspond to actual courses.
    # E.g. "sophomore standing". For Those, let's at least add a label
    for n in g.nodes():
        if 'label' not in g.node[n]:
            g.node[n]['label'] = n
    return g    
def getsatisfies(t):
    satisfies = []
    #satisfiespat = re.compile('\(A-AP|A-TH|A-AR|A-QR|D-D|D-I|D-L|ES|IE|RCH|SI|W|WI\)')
    cansatisfy = ['A-AP', 'A-TH', 'A-AR', 'A-QR', 'D-D', 'D-I', 'D-L', 'ES', 'IE', 'RCH', 'SI', 'W', 'WI']
    parenpat = re.compile('\(([^)]+)\)')
    parenmatches = parenpat.findall(t)
    if parenmatches:
        if parenmatches[-1] == 'AY':
            ay = True
            parenmatches = parenmatches[:-1]
    if parenmatches:
        if 'credits' not in parenmatches:
            thissatisfies = parenmatches[-1].replace(',','').split()
            for cs in cansatisfy:
                if cs in thissatisfies:
                    satisfies.append(cs)
    return satisfies
def getprereqs(t,pre_co='Pre'):
    prereqs = []
    prereqpat = re.compile('{pc}[-]?requisite[s]?:[^.]*'.format(pc=pre_co))
    prereqmatches = prereqpat.findall(t)
    #print(prereqmatches)
    if prereqmatches:
        # Does not deal properly with "or" or instructor permission.
        if len(prereqmatches) > 1:
            raise(Exception('Too many words "prerequisite" found'))
            #PHYS 235, MATH 320 and MATH 350
            #MATH 280 and PHYS 235
            #MATH 320 and 350
        #print("STARTING {p}".format(p=prereqmatches))
        pm = prereqmatches[0]
        pm = pm.replace('Sophomore standing or above','Sophomore/Junior/Senior standing')
        pm = pm.replace('An Interpretive Practices course or consent of the instructor','')
        pm = pm.replace('One prior philosophy course or consent of the instructor','')
        pm = pm.replace('One previous Philosophy course or consent of the instructor','')
        pm = pm.replace('Previous study in Social Sciences or Philosophy or consent of the instructor','')
        pm = pm.replace('of the','of')
        pm = pm.replace('Consent','consent')
        pm = pm.replace('consent','permission').replace('permission of instructor','')
        pm = pm.replace('either','')
        pm = pm.replace(';',',')
        pm = pm.replace('Grade of C or better in','')
        pm = pm.replace('strongly recommended','')
        pm = pm.replace(',',' and').replace(' or ',' and ')
        pm = pm.replace(' and and ',' and ') # edge
        #print("before split {s}".format(s=pm))
        pm = pm.split(' and ')
        parts = [i.strip() for i in pm if i.strip()]
        #print("Parts starts {p}".format(p=parts))
        # The first word of the first match is "Prerequisite" or something similar
        if len(parts[0].split()) > 1:
            parts[0] = ' '.join(parts[0].split()[1:])
        elif parts[0].lower() == 'prerequisite:':
            # e.g. Prerequisite: consent of instructor
            parts = []
        else:
            #print('{a} {b} {c}'.format(a=parts[0].lower(),b=))
            raise(Exception('Expected "prequisite" to start {s} got {p}'.format(s=parts[0],p=parts)))
        for (i,p) in enumerate(parts):
            courseparts = p.split()
            if len(courseparts) == 2:
                prereqs.append(p)
            elif len(courseparts) == 1:
                # assume the last one listed the dept. We may have to check two of these for, e.g.
                # Prerequisites: MATH 190, 288 and 310
                coursedept = parts[i-1].split()[0]
                if coursedept.isnumeric():
                    coursedept = parts[i-2].split()[0]
                if coursedept.isnumeric():
                    coursedept = parts[i-3].split()[0]
                prereqs.append('{d} {n}'.format(d=coursedept,n=courseparts[0]))
            else:
                if courseparts[0].lower() in ('background','faculty','an','Sophomore','one'): # e.g. 'an IP'
                    prereqs.append(' '.join(courseparts))
                else:
                    raise(Exception('Too many parts in {s}, {cp}'.format(s=prereqmatches[0],cp=courseparts)))
    return prereqs
def fudgerowpart(part,name):
    result = 'BOGUS'
    nps = name.split()
    if part == 'dept':
        if (len(nps) == 2) and nps[1].isnumeric():
            result = nps[0]
            print("\tFudging {r}".format(r=result))
            return result
        elif part in gs:
            result = part
            print("\tFudging {r}".format(r=result))
            return result
        elif 'calculus' in name.lower():
            result = 'MATH'
            print("\tFudging {r}".format(r=result))
            return result
    if part == 'num':
        if (len(nps) == 2) and nps[1].isnumeric():
            result = nps[1]
            print("\tFudging {r}".format(r=result))
            return result
    return result
def color(n):
    parts = n.split()
    p0 = parts[0]
    for (i,(dept,url)) in enumerate(deptandurl):
        if p0 == dept:
            return i
    return i+1
def plotdept(g,layout='spring',layoutparams={},drawparams={},figsize=(10,10),autocolor=False):
    fig = plt.figure(figsize=figsize)
    nxl = {'spring':nx.spring_layout,
           'shell':nx.shell_layout,
           'circular':nx.circular_layout,
           'random':nx.random_layout,
           'spectral':nx.spectral_layout,
           'graphviz':nx.graphviz_layout}[layout]
    pos=nxl(g,**layoutparams)
    if autocolor:
        node_color=[color(n) for n in g.nodes()]
        drawparams['node_color'] = node_color
    #nx.draw(G,pos, node_color = values, node_size=1500,edge_color=edge_colors,edge_cmap=plt.cm.Reds)
    #print("Draw Params {s}".format(s=drawparams))
    nx.draw_networkx(g, pos=pos,**drawparams)


In [8]:
def mgDiGcompose(g,h):
    """ Compose the graphs, but preserve as many node attributes as possible. 
    If both have the same attribute, G takes precedent.
    We will not make deepcopies of things, so if you change an attribute list, it will propagate."""
    f = nx.DiGraph()
    for hn in h.nodes():
        f.add_node(hn)
        for (hk,hv) in h.node[hn].items():
            f.node[hn][hk] = hv
    for (k1,k2) in h.edges():
        f.add_path([k1,k2])
    for gn in g.nodes():
        if gn not in f.node:
            f.add_node(gn)
        for (gk,gv) in g.node[gn].items():
            f.node[gn][gk] = gv
    for (k1,k2) in g.edges():
        f.add_path([k1,k2])
    return f
def addedges(g,h,verbose=False):
    """ If an edge in g corresoponds to nodes in h, add the edge to h. Vice versa. """
    for (k1,k2) in g.edges():
        if k1 in h.node and k2 in h.node and (k1,k2) not in h.edges():
            h.add_path([k1,k2])
            if verbose: print("added {k1} --> {k2}".format(k1=k1,k2=k2))
    for (k1,k2) in h.edges():
        if k1 in g.node and k2 in g.node and (k1,k2) not in g.edges():
            g.add_path([k1,k2])
            if verbose: print("added {k1} --> {k2}".format(k1=k1,k2=k2))
            

## Now do the following after reading the directions

OK, I lied. Making this look like a blog post was a bit more than I hoped. I think the large javascript cells look ugly and distracting. So, if you're running this interactively, FIRST run the cells above that define divs. THEN run the javascript cells below

In [9]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-BIOL.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-BIOL.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-bioheader").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-bioheader").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [10]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-BIOL.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-BIOL.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-bio").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-bio").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [11]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-PHYS.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-PHYS.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-phys").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-phys").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [12]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-MATH.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-MATH.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-math").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-math").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});

<IPython.core.display.Javascript object>

In [13]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-CHEM.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-CHEM.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-chem").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-chem").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [14]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-GEOL.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-GEOL.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-geol").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-geol").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [15]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-CS.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-CS.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-cs").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-cs").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [16]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes-BIOC.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links-BIOC.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 800,
                height = 800;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-bioc").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-bioc").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});


<IPython.core.display.Javascript object>

In [17]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.

    d3.tsv('curriculum-nodes.tsv', function(error,cnodes) {
        
        var nodes = {}
        cnodes.forEach( function(cnode) {
            nodes[cnode.label] = {name: cnode.label,
                                dept: cnode.dept,
                                };
            }
            );

        // get the data
        d3.csv("curriculum-links.csv", function(error, links) {

            //var nodes = {};

            // Compute the distinct nodes from the links.
            links.forEach(function(link) {
                link.source = nodes[link.source] || 
                    (nodes[link.source] = {name: link.source, dept: link.source.replace(/[0-9]+/g,'')  });
                link.target = nodes[link.target] || 
                    (nodes[link.target] = {name: link.target, dept: link.target.replace(/[0-9]+/g,'')  });
                link.value = +link.value;
            });

                //nodes["COW"] = {name: "COW",dept: "FARM"};

            var width = 1400,
                height = 1400;

            // We create a color scale.
            var color = d3.scale.category10();

            var force = d3.layout.force()
                .nodes(d3.values(nodes))
                .links(links)
                .size([width, height])
                .linkDistance(60)
                .charge(-300)
                .on("tick", tick)
                .start();

            var svg = d3.select("#d3-scidiv").select("svg")

                if (svg.empty()) {
                    svg = d3.select("#d3-scidiv").append("svg")
                                .attr("width", width)
                                .attr("height", height);
                }


            // build the arrow.
            svg.append("svg:defs").selectAll("marker")
                .data(["end"])      // Different link/path types can be defined here
              .enter().append("svg:marker")    // This section adds in the arrows
                .attr("id", String)
                .attr("viewBox", "0 -5 10 10")
                .attr("refX", 15)
                .attr("refY", -1.5)
                .attr("markerWidth", 6)
                .attr("markerHeight", 6)
                .attr("orient", "auto")
              .append("svg:path")
                .attr("d", "M0,-5L10,0L0,5");

            // add the links and the arrows
            var path = svg.append("svg:g").selectAll("path")
                .data(force.links())
              .enter().append("svg:path")
            //    .attr("class", function(d) { return "link " + d.type; })
                .attr("class", "link")
                .attr("marker-end", "url(#end)");

            // define the nodes
            var node = svg.selectAll(".node")
                .data(force.nodes())
              .enter().append("g")
                .attr("class", "node")
                .on("click", click)
                .on("dblclick", dblclick)
                .call(force.drag);

            // add the nodes
            node.append("circle")
                .attr("r", 5)
                .style("fill", function(d) { return color(d.dept); });

            // add the text 
            node.append("text")
                .attr("x", 12)
                .attr("dy", ".35em")
                //.style("fill", "black")
                //.style("color","black")
                .text(function(d) { return d.name; });

            node.append("title")
                        .text(function(d) { return d.label; });


            // add the curvy lines
            function tick() {
                path.attr("d", function(d) {
                    var dx = d.target.x - d.source.x,
                        dy = d.target.y - d.source.y,
                        dr = Math.sqrt(dx * dx + dy * dy);
                    return "M" + 
                        d.source.x + "," + 
                        d.source.y + "A" + 
                        dr + "," + dr + " 0 0,1 " + 
                        d.target.x + "," + 
                        d.target.y;
                });

                node
                    .attr("transform", function(d) { 
                return "translate(" + d.x + "," + d.y + ")"; });
            }

            // action to take on mouse click
            function click() {
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 22)
                    //.style("fill", "steelblue")
                    //.style("stroke", "lightsteelblue")
                    .style("stroke-width", ".5px")
                    .style("font", "20px sans-serif");
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 16)
                    //.style("fill", "lightsteelblue")
                    ;
            }

            // action to take on mouse double click
            function dblclick() {
                d3.select(this).select("circle").transition()
                    .duration(750)
                    .attr("r", 6)
                    //.style("fill", "#ccc")
                    ;
                d3.select(this).select("text").transition()
                    .duration(750)
                    .attr("x", 12)
                    .style("stroke", "none")
                    .style("fill", "black")
                    .style("stroke", "none")
                    .style("font", "10px sans-serif");
            }

            });



    }
    );
    
});

<IPython.core.display.Javascript object>

In [18]:
%%javascript
// We load the d3.js library from the Web.
require.config({paths: {d3: "http://d3js.org/d3.v3.min"}});
require(["d3"], function(d3) {
    // The code in this block is executed when the 
    // d3.js library has been loaded.
    
    // First, we specify the size of the canvas containing
    // the visualization (size of the <div> element).
    var width = 1200,
        height = 1000;

    // We create a color scale.
    var color = d3.scale.category10();

    // We create a force-directed dynamic graph layout.
    var force = d3.layout.force()
        .charge(-120)
        .linkDistance(30)
        .size([width, height]);

    // In the <div> element, we create a <svg> graphic
    // that will contain our interactive visualization.
    var svg = d3.select("#d3-example1").select("svg")
    if (svg.empty()) {
        svg = d3.select("#d3-example1").append("svg")
                    .attr("width", width)
                    .attr("height", height);
    }
        
    // We load the JSON file.
    d3.json("curriculum.json", function(error, graph) {
        // In this block, the file has been loaded
        // and the 'graph' object contains our graph.
        
        // We load the nodes and links in the force-directed
        // graph.
        force.nodes(graph.nodes)
            .links(graph.links)
            .start();

  var link = svg.selectAll(".link")
      .data(graph.links)
    .enter().append("line")
      .attr("class", "link")
      .style("stroke-width", function(d) { return Math.sqrt(d.value); });

  var gnodes = svg.selectAll('g.gnode')
     .data(graph.nodes)
     .enter()
     .append('g')
     .classed('gnode', true);
    
  var node = gnodes.append("circle")
      .attr("class", "node")
      .attr("r", 7)
      .style("fill", function(d) { return color(d.dept); })
      .call(force.drag);

  var labels = gnodes.append("text")
      .text(function(d) { return d.label; })
      .attr("font-family", "sans-serif")
      .attr("font-size", "15px");
       //.attr("fill", "red");
        
        // The name of each node is the node number.
  var titles = gnodes.append("title")
            .text(function(d) { return d.label; });
        

  console.log(labels);
    
  force.on("tick", function() {
    link.attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    gnodes.attr("transform", function(d) { 
        return 'translate(' + [d.x, d.y] + ')'; 
    });

            
        });
    });
});


<IPython.core.display.Javascript object>

# Comments!

Let's try just sticking the Disqus code here

In [19]:
%%html
    <div id="disqus_thread"></div>
    <script type="text/javascript">
        /* * * CONFIGURATION VARIABLES: EDIT BEFORE PASTING INTO YOUR WEBPAGE * * */
        var disqus_shortname = 'biophysicsandbeer'; // required: replace example with your forum shortname

        /* * * DON'T EDIT BELOW THIS LINE * * */
        (function() {
            var dsq = document.createElement('script'); dsq.type = 'text/javascript'; dsq.async = true;
            dsq.src = '//' + disqus_shortname + '.disqus.com/embed.js';
            (document.getElementsByTagName('head')[0] || document.getElementsByTagName('body')[0]).appendChild(dsq);
        })();
    </script>
    <noscript>Please enable JavaScript to view the <a href="http://disqus.com/?ref_noscript">comments powered by Disqus.</a></noscript>
    